In [1]:
import matplotlib.pyplot as plt
import requests
import numpy as np
import pandas as pd
import json
import operator
from tqdm import tqdm_notebook as tqdm
from pprint import pprint

## Reading & Cleaning Carriers Data

In [2]:
#Reading carriers data csv 
carriers_data = "Resources/Test_carriers_df.csv"
carriers_data_df = pd.read_csv(carriers_data,encoding="utf-8")
carriers_data_df

#This is the dictionary
airlines_dict = {row[0]: row[1] for k, row in carriers_data_df.iterrows()}


In [3]:
#Renaming columns
#https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas
carriers_data_df = carriers_data_df.rename(columns={'CarrierId': 'Airline ID', 'Name':'Airline Name'})

convert_dict2 = {'Airline ID': object}
carriers_data_df = carriers_data_df.astype(convert_dict2) 

#Expport clean data frame to csv
export_csv = carriers_data_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\carriersCLEAN.csv',index = False)

carriers_data_df.head()

,Airline ID,Airline Name
0,835,Air Canada
1,898,EVA Air
2,929,Air China
3,1065,Frontier Airlines
4,1276,Japan Airlines


## Reading & Cleaning Quotes Data

In [4]:
#Reading Quoutes data csv
quotes_data = "Resources/Test_merged_quotes_outbound.csv"
quotes_data_df = pd.read_csv(quotes_data,encoding="utf-8")
quotes_data_df.head()

,Direct,MinPrice,OutboundLeg,QuoteDateTime,QuoteId,CarrierIds,DepartureDate,DestinationId,OriginId
0,False,454.0,"{'CarrierIds': [1276], 'OriginId': 58440, 'Des...",2019-07-22T01:39:00,1,[1276],2019-12-01T00:00:00,56615,58440
1,False,487.0,"{'CarrierIds': [1467], 'OriginId': 58440, 'Des...",2019-07-24T23:59:00,2,[1467],2019-12-04T00:00:00,56615,58440
2,False,684.0,"{'CarrierIds': [1065], 'OriginId': 58440, 'Des...",2019-07-22T10:14:00,3,[1065],2019-12-12T00:00:00,56615,58440
3,False,918.0,"{'CarrierIds': [898], 'OriginId': 58440, 'Dest...",2019-07-22T10:14:00,4,[898],2019-12-14T00:00:00,56615,58440
4,False,1022.0,"{'CarrierIds': [1276], 'OriginId': 58440, 'Des...",2019-07-25T14:55:00,5,[1276],2019-12-20T00:00:00,56615,58440


In [5]:
#Delete columns that we don't use
#https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/
quotes_data_df = quotes_data_df.drop(['OutboundLeg','QuoteDateTime'], axis =1)

#Delete the brackets from carrier Ids
quotes_data_df['CarrierIds'] = quotes_data_df['CarrierIds'].str.replace("[","")
quotes_data_df['CarrierIds'] = quotes_data_df['CarrierIds'].str.replace("]","")

#Delete the time from the daparture date
quotes_data_df['DepartureDate'] = quotes_data_df['DepartureDate'].str.replace("T00:00:00","")

#Change order of columns 
#https://erikrood.com/Python_References/change_order_dataframe_columns_final.html
quotes_data_df = quotes_data_df[['OriginId','DestinationId','MinPrice','CarrierIds','Direct','DepartureDate','QuoteId']]

#Renaming columns
#https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas
quotes_data_df = quotes_data_df.rename(columns={'OriginId': 'City ID Departure', 'DestinationId':'City ID','MinPrice':'Min Price', 'CarrierIds': 'Airline ID', 'Direct': 'Non-Stop Flight', 'DepartureDate':'Departure Date', 'QuoteId': 'Quote ID'})

#Change Airline ID to numeric
#https://www.geeksforgeeks.org/change-data-type-for-one-or-more-columns-in-pandas-dataframe/
convert_dict = {'Airline ID': int}
quotes_data_df = quotes_data_df.astype(convert_dict) 


#Expport clean data frame to csv
export_csv = quotes_data_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\quotesCLEAN.csv',index = False)



## Reading & Cleaning Places data

In [6]:
#Reading places data csv
places_data = "Resources/Testplacescsv.csv"
places_data_df = pd.read_csv(places_data,encoding="utf-8")
places_data_df.head()

,CityId,CityName,CountryName,IataCode,Name,PlaceId,SkyscannerCode,Type
0,HKGA,Hong Kong,Hong Kong,HKG,Hong Kong International,56615,HKG,Station
1,HOUA,Houston,United States,IAH,Houston George Bush Intercntl.,58440,IAH,Station
2,BKKT,Bangkok,Thailand,BKK,Bangkok Suvarnabhumi,42795,BKK,Station
3,HOUA,Houston,United States,IAH,Houston George Bush Intercntl.,58440,IAH,Station
4,HOUA,Houston,United States,IAH,Houston George Bush Intercntl.,58440,IAH,Station


In [7]:
#Drop/remove entire column CityId, Type, drop SkyscannerCode, change PlaceId to string
#Delete columns that we don't use
#https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/
places_data_df = places_data_df.drop(['CityId','Type','SkyscannerCode'], axis =1)

In [8]:
#Change order of columns 
#https://erikrood.com/Python_References/change_order_dataframe_columns_final.html
places_data_df = places_data_df[['PlaceId','CityName','Name','IataCode','CountryName']]

#Renaming columns
#https://stackoverflow.com/questions/11346283/renaming-columns-in-pandas
places_data_df = places_data_df.rename(columns={'CityName': 'City Name', 'CountryName':'Country Name','IataCode':'Iata Code', 'Name': 'Airport Name', 'PlaceId': 'City ID'})

#Export clean data frame to csv
export_csv = places_data_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\placesCLEAN.csv',index = False)

In [9]:
places_data_df.head()

,City ID,City Name,Airport Name,Iata Code,Country Name
0,56615,Hong Kong,Hong Kong International,HKG,Hong Kong
1,58440,Houston,Houston George Bush Intercntl.,IAH,United States
2,42795,Bangkok,Bangkok Suvarnabhumi,BKK,Thailand
3,58440,Houston,Houston George Bush Intercntl.,IAH,United States
4,58440,Houston,Houston George Bush Intercntl.,IAH,United States


## Merging 

In [10]:
merge_1 = pd.merge(places_data_df, quotes_data_df, how = 'outer', on = 'City ID' )
merge_1.head(1000)

,City ID,City Name,Airport Name,Iata Code,Country Name,City ID Departure,Min Price,Airline ID,Non-Stop Flight,Departure Date,Quote ID
0,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,454.0,1276.0,False,2019-12-01,1.0
1,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,487.0,1467.0,False,2019-12-04,2.0
2,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,684.0,1065.0,False,2019-12-12,3.0
3,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,918.0,898.0,False,2019-12-14,4.0
4,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,1022.0,1276.0,False,2019-12-20,5.0
5,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,500.0,835.0,False,2019-12-25,6.0
6,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,674.0,898.0,False,2019-12-26,7.0
7,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,619.0,835.0,False,2019-12-27,8.0
8,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,487.0,929.0,False,2019-12-31,9.0
9,58440,Houston,Houston George Bush Intercntl.,IAH,United States,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#Drop NA values
#http://www.datasciencemadesimple.com/drop-rows-with-nan-na-drop-missing-value-in-pandas-python-2/
merge_1 = merge_1.dropna()
merge_1



,City ID,City Name,Airport Name,Iata Code,Country Name,City ID Departure,Min Price,Airline ID,Non-Stop Flight,Departure Date,Quote ID
0,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,454.0,1276.0,False,2019-12-01,1.0
1,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,487.0,1467.0,False,2019-12-04,2.0
2,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,684.0,1065.0,False,2019-12-12,3.0
3,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,918.0,898.0,False,2019-12-14,4.0
4,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,1022.0,1276.0,False,2019-12-20,5.0
5,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,500.0,835.0,False,2019-12-25,6.0
6,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,674.0,898.0,False,2019-12-26,7.0
7,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,619.0,835.0,False,2019-12-27,8.0
8,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,487.0,929.0,False,2019-12-31,9.0
56,42795,Bangkok,Bangkok Suvarnabhumi,BKK,Thailand,58440.0,494.0,1713.0,False,2019-12-01,1.0


In [12]:
# Merge Airline Name, keep in mind that a dictionary has been created for the data frame of carriers to eliminate duplicates
#https://stackoverflow.com/questions/20250771/remap-values-in-pandas-column-with-a-dict

# Map the city name to the Airline ID by creating a new Airline Name column
merge_1['Airline Name'] = merge_1['Airline ID'].map(airlines_dict)
merge_1

,City ID,City Name,Airport Name,Iata Code,Country Name,City ID Departure,Min Price,Airline ID,Non-Stop Flight,Departure Date,Quote ID,Airline Name
0,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,454.0,1276.0,False,2019-12-01,1.0,Japan Airlines
1,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,487.0,1467.0,False,2019-12-04,2.0,Spirit Airlines
2,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,684.0,1065.0,False,2019-12-12,3.0,Frontier Airlines
3,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,918.0,898.0,False,2019-12-14,4.0,EVA Air
4,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,1022.0,1276.0,False,2019-12-20,5.0,Japan Airlines
5,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,500.0,835.0,False,2019-12-25,6.0,Air Canada
6,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,674.0,898.0,False,2019-12-26,7.0,EVA Air
7,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,619.0,835.0,False,2019-12-27,8.0,Air Canada
8,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,487.0,929.0,False,2019-12-31,9.0,Air China
56,42795,Bangkok,Bangkok Suvarnabhumi,BKK,Thailand,58440.0,494.0,1713.0,False,2019-12-01,1.0,Singapore Airlines


In [14]:
#Drop duplicates cities such as Johannesburg
merge_1 = merge_1.drop_duplicates()
#Export and save to CSV
export_csv = merge_1.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\MergeCleanFINAL.csv',index = False)


#print(Houston_merge_1.dtypes)

merge_1


,City ID,City Name,Airport Name,Iata Code,Country Name,City ID Departure,Min Price,Airline ID,Non-Stop Flight,Departure Date,Quote ID,Airline Name
0,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,454.0,1276.0,False,2019-12-01,1.0,Japan Airlines
1,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,487.0,1467.0,False,2019-12-04,2.0,Spirit Airlines
2,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,684.0,1065.0,False,2019-12-12,3.0,Frontier Airlines
3,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,918.0,898.0,False,2019-12-14,4.0,EVA Air
4,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,1022.0,1276.0,False,2019-12-20,5.0,Japan Airlines
5,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,500.0,835.0,False,2019-12-25,6.0,Air Canada
6,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,674.0,898.0,False,2019-12-26,7.0,EVA Air
7,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,619.0,835.0,False,2019-12-27,8.0,Air Canada
8,56615,Hong Kong,Hong Kong International,HKG,Hong Kong,58440.0,487.0,929.0,False,2019-12-31,9.0,Air China
56,42795,Bangkok,Bangkok Suvarnabhumi,BKK,Thailand,58440.0,494.0,1713.0,False,2019-12-01,1.0,Singapore Airlines


In [ ]:
################ Reading CSV to combined everything in one table ##################################

#Reading data csv 
Houston_data = "Resources/Houston Data/HOU_MergeCleanFINAL07282019.csv"
Houston_data = pd.read_csv(Houston_data,encoding="utf-8")

Chicago_data = "Resources/Chicago Data/CHI_MergeCleanFINAL07282019.csv"
Chicago_data = pd.read_csv(Chicago_data,encoding="utf-8")

Atlanta_data = "Resources/Atlanta Data/ATL_MergeCleanFINAL07282019.csv"
Atlanta_data = pd.read_csv(Atlanta_data,encoding="utf-8")

NY_data = "Resources/NY Data/NY_MergeCleanFINAL07282019.csv"
NY_data = pd.read_csv(NY_data,encoding="utf-8")

Seattle_data = "Resources/Seattle Data/SE_MergeCleanFINAL07282019.csv"
Seattle_data = pd.read_csv(Seattle_data,encoding="utf-8")

LA_data = "Resources/Los Angeles Data/LA_MergeCleanFINAL07282019.csv"
LA_data = pd.read_csv(LA_data,encoding="utf-8")


In [ ]:
############################ Create one big combined table with all data #####################
append10 = Houston_data.append(Atlanta_data)
append20 = append10.append(NY_data)
append30 = append20.append(LA_data)
append40 = append30.append(Seattle_data)
combined_table = append40.append(Chicago_data)
#combined_table

In [ ]:
export_csv = combined_table.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\TestDataCLEANDate.csv',index = False)
